# Proceso iterativo para definir y modelar sistemas

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

Como ejemplo, revisaremos el sistema de bicicletas compartidas de la clase anterior, consideraremos sus fortalezas y debilidades y lo mejoraremos gradualmente. También veremos formas de usar el modelo para comprender el comportamiento del sistema y evaluar diseños destinados a que funcione mejor.

## Metodologia Iterativa

El sistema de bicicletas compartidas se modelo la clase pasada de forma extremadamente simple, pero se basa en supuestos poco realistas. Antes de continuar, tómate un momento para analizar todos los componentes que puede tener este sistema. ¿En qué supuestos se basa? Haz una lista de formas en que este modelo podría ser poco realista; es decir, ¿cuáles son las diferencias entre el modelo y el sistema real?

Aquí están algunas de las diferencias de mi lista:

* En el modelo, un estudiante tiene la misma probabilidad de llegar en cualquier período de un minuto. En realidad, esta probabilidad varía según la hora del día, el día de la semana, etc.

* El modelo no tiene en cuenta el tiempo de viaje de una estación de bicicletas a otra.

* El modelo no verifica si una bicicleta está disponible, por lo que es posible que el número de bicicletas sea negativo (como habrás notado en algunas de tus simulaciones).


Para obtener un modelo y por consiguiente una simulación que entregue resultados adecuados es necesario definir correctamente el sistema y realizar algunas supocisiones iniciales. Algunas de estas suposiciones de modelado son mejores que otras. Por ejemplo, el primer supuesto podría ser razonable si simulamos el sistema por un período corto de tiempo, como una hora.

El segundo supuesto no es muy realista, pero podría no afectar demasiado los resultados, dependiendo de para qué usemos el modelo. Por otro lado, el tercer supuesto parece más problemático. Sin embargo, es relativamente fácil de corregir; en este capítulo lo solucionaremos.

Este proceso, que consiste en comenzar con un modelo simple, identificar los problemas más importantes y realizar mejoras graduales, se llama *metodologia iterativa para definir y modelar sistemas*.

Para cualquier sistema físico o proceso real, existen muchos modelos posibles, basados en distintos supuestos y simplificaciones. A menudo se necesitan varias iteraciones para desarrollar un modelo que sea lo suficientemente bueno para el propósito previsto, pero que no sea más complicado de lo necesario.


## Más de un Objeto de Estado

Realicemos algunos cambios al código de la clase anterior. Primero, generalizaré las funciones que escribimos para que tomen un objeto `State` como parámetro.
Luego, hagamos que el código sea más legible agregando documentación.

Aquí está una de las funciones del capítulo anterior, `bicicleta_a_cata`:


In [ ]:
def bicicleta_a_c4ta():
    bikeshare_state.robledo -= 1
    bikeshare_state.c4ta += 1

Cuando se llama a esta función, modifica `bikeshare_state`. Mientras solo haya un objeto `State`, eso está bien, pero ¿qué pasa si existe más de un sistema de bicicletas compartidas? ¿O si queremos ejecutar más de una simulación?

Esta función es más flexible si le ponemos un objeto `estado` como parámetro. 


In [ ]:
def Estado(**variables):
    """Contiene los valores de las variables de estado.

    Args:
        **variables: Argumentos con nombre para almacenar como variables de estado.

    Returns:
        pd.Series: Serie con las variables de estado.
    """
    return pd.Series(variables, name="state")


Y así es como se vería:

In [ ]:
def bicicleta_a_c4ta(estado):
    estado.robledo -= 1
    estado.c4ta += 1

El nombre del parámetro es `estado`, en lugar de `bikeshare_state`, ya que el valor de `estado` podría ser cualquier objeto `Estado`, no solo el que llamamos `bikeshare_state`.

Esta versión de `bicicleta_a_c4ta` requiere un objeto `Estado` como parámetro, por lo que debemos proporcionar uno cuando la llamemos:


In [ ]:
bikeshare_state = Estado(robledo=10, c4ta=2)
bicicleta_a_c4ta(bikeshare_state)

Nuevamente, el argumento que proporcionamos se asigna al parámetro, por lo que esta llamada a la función tiene el mismo efecto que:

```python
estado = bikeshare_state
estado.robledo -= 1 
estado.cata += 1
```

Ahora podemos crear tantos objetos `Estado` como queramos:


In [ ]:
bikeshare_state1 = Estado(robledo=10, c4ta=2)
bikeshare_state2 = Estado(robledo=2, c4ta=10)

y así poder actualizarlos de forma independiente

In [ ]:
bicicleta_a_c4ta(bikeshare_state1)
bicicleta_a_c4ta(bikeshare_state2)

Los cambios en `bikeshare_state1` no afectan a `bikeshare_state2`, y viceversa. De este modo, podemos simular distintos sistemas de bicicletas compartidas o ejecutar múltiples simulaciones del mismo sistema.


## Documentación

Otro problema con el código que tenemos hasta ahora es que no contiene *documentación*.
La documentación es texto que añadimos a un programa para ayudar a otros programadores a leerlo y entenderlo. No tiene ningún efecto en el programa cuando se ejecuta.

Existen dos tipos de documentación: *docstrings* y *comentarios*:

* Un docstring es una cadena entre comillas triples que aparece al inicio de una función.

* Un comentario es una línea de texto que comienza con un símbolo de numeral, `#`.

Aquí tienes una versión de `bicicleta_a_robledo` con un docstring y un comentario.

In [ ]:
def bicicleta_a_robledo(estado):
    """Mover una bicicleta de Wellesley a Olin.
    
    state: objeto State de bikeshare
    """
    # Disminuimos una variable de estado y aumentamos la
    # otra para que el número total de bicicletas no cambie.
    estado.robledo += 1
    estado.c4ta -= 1


Los docstrings siguen un formato convencional:

* La primera línea es una sola oración que describe lo que hace la función.

* Las líneas siguientes explican cuáles son los parámetros.

El docstring de una función debe incluir la información que alguien necesita saber para *usar* la función; no debe incluir detalles sobre cómo funciona la función.

Los comentarios proporcionan detalles sobre cómo funciona la función, especialmente si hay algo que no sería obvio para alguien que esté leyendo el programa.

## Bicicletas Negativas

Los cambios que hemos hecho hasta ahora mejoran la calidad del código, pero no hemos hecho nada para mejorar la calidad del modelo. Hagámoslo ahora.

Actualmente, la simulación no verifica si hay una bicicleta disponible cuando llega un cliente, por lo que el número de bicicletas en una ubicación puede ser negativo. Eso no se ajusta al sistema dado que nunca puede haber un numero de bicicletas negativo.

Aquí tienes una versión de `bicicleta_a_robledo` que corrige el problema:

In [ ]:
def bicicleta_a_robledo(estado):
    if estado.c4ta == 0:
        return
    estado.c4ta -= 1
    estado.robledo += 1

La primera línea verifica si el número de bicicletas en **C4TA** es cero. Si es así, utiliza una *sentencia return*, lo que hace que la función termine de inmediato, sin ejecutar el resto de las instrucciones. Por lo tanto, si no hay bicicletas en **C4TA**, salimos de `bicicleta_a_robledo` sin cambiar el estado.

Podemos probarlo inicializando el estado sin bicicletas en **C4TA** y llamando a `bicicleta_a_robledo`.

In [ ]:
bikeshare_state = Estado(robledo=12, c4ta=0)
bicicleta_a_robledo(bikeshare_state)

El estado del sistema no deberia cambiar, lograndose asi que nunca halla un número de bicicletas negativas

In [ ]:
print(bikeshare_state)

## Operadores de Comparación

La versión actualizada de `bicicleta_a_robledo` usa el operador de igualdad, `==`, que compara dos valores y devuelve `True` si son iguales, y `False` en caso contrario.

Es fácil confundir el operador de igualdad con el operador de asignación, `=`, que asigna un valor a una variable. Por ejemplo, la siguiente instrucción crea una variable `x`, si no existe ya, y le asigna el valor `5`.


In [ ]:
x = 8

Por otro lado, la siguiente instrucción verifica si `x` es `5` y devuelve `True` o `False`. No crea `x` ni cambia su valor.


In [ ]:
x == 8

Pueden usar el operador de igualdad en una instrucción `if`, de esta manera:

In [ ]:
if x == 8:
    print('Si, x es 8')

El operador de igualdad es uno de los *operadores de comparación* de Python; la lista completa está en la siguiente tabla.

| Operación              | Símbolo |
| ---------------------- | ------- |
| Menor que              | `<`     |
| Mayor que              | `>`     |
| Menor o igual que      | `<=`    |
| Mayor o igual que      | `>=`    |
| Igual                  | `==`    |
| Distinto de / No igual | `!=`    |

## Métricas

Volviendo al sistema de bicicletas compartidas, en este punto tenemos la capacidad de simular el comportamiento del sistema. Como la llegada de los clientes es aleatoria, el estado del sistema es diferente cada vez que ejecutamos una simulación. Sistemas como este se llaman aleatorios o *estocásticos*; los sistemas que hacen lo mismo cada vez que se ejecutan son *deterministas*.

Supongamos que queremos usar nuestro modelo para predecir qué tan bien funcionará el sistema de bicicletas compartidas, o para diseñar un sistema que funcione mejor. Primero, tenemos que decidir qué queremos decir con “qué tan bien” y “mejor”.

Desde el punto de vista del cliente, podríamos querer conocer la probabilidad de encontrar una bicicleta disponible. Desde el punto de vista del propietario del sistema, podríamos querer minimizar el número de clientes que no consiguen una bicicleta cuando la desean, o maximizar el número de bicicletas en uso. A las estadísticas como estas, que cuantifican qué tan bien funciona el sistema, se les llama *métricas*.

Como ejemplo, midamos el número de clientes insatisfechos.
Aquí tienes una versión de `bicicleta_a_robledo` que lleva el registro del número de clientes que llegan a una estación sin bicicletas:


In [ ]:
def bicicleta_a_robledo(estado):
    if estado.c4ta == 0:
        estado.c4ta_vacia += 1
        return
    estado.c4ta -= 1
    estado.robledo += 1

Si un cliente llega a la estación de **C4TA** y no encuentra una bicicleta disponible, `bicicleta_a_robledo` actualiza `c4ta_vacia`, que cuenta el número de clientes insatisfechos.

Esta función solo funciona si inicializamos `c4ta_vacia` cuando creamos el objeto `Estado`, de esta manera:


In [ ]:
bikeshare_state = Estado(robledo=12, c4ta=0, 
                  c4ta_vacia=0)

Podemos problarlo al llamar `bicicleta_a_robledo`:

In [ ]:
bicicleta_a_robledo(bikeshare_state)

Después de esta actualización, debería haber 12 bicicletas en **Robledo**, ninguna bicicleta en **C4TA** y un cliente insatisfecho.

In [ ]:
print(bikeshare_state)

## Resumen

hasta este momento, hemos escrito varias versiones de `bicicleta_a_robledo`:

* Agregamos un parámetro, `estado`, para poder trabajar con más de un objeto `Estado`.

* Añadimos un *docstring* que explica cómo usar la función y un comentario que explica cómo funciona.

* Usamos un operador condicional, `==`, para verificar si hay una bicicleta disponible y así evitar bicicletas negativas.

* Agregamos una variable de estado, `c4ta_empty`, para contar el número de clientes insatisfechos, que es una métrica que utilizaremos para cuantificar qué tan bien funciona el sistema.

En los ejercicios, actualizarás `bicicleta_a_c4ta` de la misma manera y lo probarás ejecutando una simulación.

Aquí está el código que tenemos hasta ahora, con *docstrings*, todo en un solo lugar:


In [ ]:
def correr_simulacion(estado, p1, p2, num_pasos):
    """Simular el número dado de pasos de tiempo.
    
    state: objeto State
    p1: probabilidad de llegada de un cliente a Robledo->C4TA
    p2: probabilidad de llegada de un cliente a C4TA->Robledo
    num_pass: número de pasos de tiempo
    """
    results = TimeSeries()
    results[0] = estado.robledo
    
    for i in range(num_pasos):
        paso(state, p1, p2)
        results[i+1] = estado.robledo
        
    results.plot(label='Robledo')
    decorar(title='Sistema de Bicicletas Compartidas Robledo-C4TA',
             xlabel='Paso de tiempo (min)', 
             ylabel='Número de bicicletas')
    
def paso(estado, p1, p2):
    """Simular un paso de tiempo.
    
    state: objeto State del sistema de bicicletas compartidas
    p1: probabilidad de un viaje Robledo->C4TA
    p2: probabilidad de un viaje C4TA->Robledo
    """
    if lanzamiento(p1):
        bike_to_c4ta(estado)
    
    if lanzamiento(p2):
        bike_to_robledo(estado)


def bike_to_robledo(estado):
    """Mover una bicicleta de C4TA a Robledo.
    
    state: objeto State del sistema de bicicletas compartidas
    """
    if estado.c4ta == 0:
        estado.c4ta_vacia += 1
        return
    estado.c4ta -= 1
    estado.robledo += 1


def bike_to_c4ta(estado):
    """Mover una bicicleta de Robledo a C4TA.
    
    state: objeto State del sistema de bicicletas compartidas
    """
    estado.robledo -= 1
    estado.c4ta += 1

def lanzamiento(p=0.5):
    """Lanza una moneda con la probabilidad dada.

    Args:
        p (float): Probabilidad entre 0 y 1.

    Returns:
        bool: True o False.
    """
    return np.random.random() < p

def TimeSeries(*args, **kwargs):
    """Crear un objeto pd.Series para representar una serie temporal.

    Args:
        *args: Argumentos pasados a pd.Series.
        **kwargs: Argumentos con nombre pasados a pd.Series.

    Returns:
        pd.Series: Serie con nombre de índice 'Time' y nombre 'Quantity'.
    """
    if args or kwargs:
        underride(kwargs, dtype=float)
        series = pd.Series(*args, **kwargs)
    else:
        series = pd.Series([], dtype=float)

    series.index.name = "Time"
    if "name" not in kwargs:
        series.name = "Quantity"
    return series

def underride(d, **options):
    """Agregar pares clave-valor a d solo si la clave no está en d.

    Si d es None, crear un nuevo diccionario.

    Args:
        d (dict): Diccionario a actualizar.
        **options: Argumentos con nombre para agregar a d.

    Returns:
        dict: Diccionario actualizado.
    """
    if d is None:
        d = {}

    for key, val in options.items():
        d.setdefault(key, val)

    return d

def decorar(**options):
    """Decora los ejes actuales.

    Llama a decorar con argumentos con nombre, por ejemplo:
    decorar(title='Título',
                xlabel='x',
                ylabel='y')

    Los argumentos con nombre pueden ser cualquiera de las propiedades de los ejes:
    https://matplotlib.org/api/axes_api.html

    Args:
        **options: Argumentos con nombre para las propiedades de los ejes.
    """
    ax = plt.gca()
    ax.set(**options)

    handles, labels = ax.get_legend_handles_labels()
    if handles:
        ax.legend(handles, labels)

    plt.tight_layout()

def Estado(**variables):
    """Contiene los valores de las variables de estado.

    Args:
        **variables: Argumentos con nombre para almacenar como variables de estado.

    Returns:
        pd.Series: Serie con las variables de estado.
    """
    return pd.Series(variables, name="state")


### Ejercicio 1

Modifica `bicicleta_a_c4ta` para que verifique si hay una bicicleta disponible en **Robledo**.
Si no la hay, debe sumar `1` a `robledo_vacia`.

Para probarlo, crea un `Estado` que inicialice `robledo` y `robledo_vacia` en `0`, ejecuta `bicicleta_a_c4ta` y revisa el resultado.


In [ ]:
# Solución aqui

### Ejercicio 2

Ahora ejecuta la simulación con los parámetros `p1=0.3`, `p2=0.2` y `num_pasos=60`, y confirma que el número de bicicletas nunca sea negativo.

Comienza con este estado inicial:


In [ ]:
bikeshare_state = Estado(robledo=20, c4ta=10,
                  robledo_vacia=0, c4ta_vacia=0)

In [ ]:
# Solución aqui